# # Tutorials-01/ How do I use these QE tutorials in Jupyter Notebook?!



***

## Description:
<a id="Sec1"></a>
This script sets up a non-self-consistent field (NSCF) calculation for computing the Density of States (DOS) of a silicon crystal. The calculation uses a 6x6x6 k-point grid for sampling the Brillouin zone, a Gaussian smearing method for partial occupancies, and an energy cutoff of 40 Ry for the wavefunction.

In [ ]:
##  Step-by-Step for NSCF Calculation Input File Preparation:

In [ ]:
Prepare the nscf Input File:
Modify the &system card in the input file to include the occupations parameter as tetrahedra (a suitable method for DOS calculations).
Update the k-point grid to a 12 × 12 × 12 grid using the automatic option for better sampling of the Brillouin zone.
Add nosym = .TRUE. to avoid the generation of additional k-points in low symmetry cases.
Ensure Consistency with Previous SCF Calculation:
The outdir and prefix parameters must remain the same as in the previous SCF step, as some inputs and outputs from the SCF calculation are required for the NSCF calculation.
Specify Larger Number of Bands (nbnd):
To account for unoccupied bands, specify a larger number for the nbnd parameter.
The number of occupied bands can be found in the SCF output as the number of Kohn-Sham states.

In [4]:
%%writefile nscf_si.in
######################################
&control
    calculation = 'nscf'
    prefix = 'silicon_dos'
    outdir = './tmp/'
    pseudo_dir = './pseudos/'
    tprnfor = .true.
    tstress = .true.
/

&system
    ibrav = 2
    celldm(1) = 14
    nat = 2
    ntyp = 1
    ecutwfc = 40
    ecutrho = 400
# Method for DOS calculation
    occupations = 'tetrahedra'           
    smearing = 'gaussian'
    degauss = 0.02
    nbnd = 200                          ! Specify a larger number of bands for unoccupied states
    nosym = .TRUE.                       ! Avoid generation of additional k-points in low symmetry cases
/

&electrons
    conv_thr = 1e-8
    mixing_beta = 0.7
    diagonalization = 'david'
/

&ions
    ion_dynamics = 'none'
/

&cell
    cell_dofree = 'none'
/

ATOMIC_SPECIES
  Si  28.0855  Si.pz-vbc.UPF

ATOMIC_POSITIONS (alat)
  Si  0.00  0.00  0.00
  Si  0.25  0.25  0.25

K_POINTS (automatic)
  12 12 12 1 1 1                         ! 12x12x12 k-point grid
# End of Script


Overwriting nscf_si.in


***
## Step 2: Run in QE 


In [6]:
## Run pw.x in non-self consistent mode for silicon
!pw.x < pw.nscf.silicon_dos.in > pw.nscf.silicon_dos.out

Le fichier spécifié est introuvable.


In [ ]:
he DOS input file as follows

In [8]:
&DOS
  prefix='silicon',
  outdir='./tmp/',
  fildos='si_dos.dat',
  emin=-9.0,
  emax=16.0
/

SyntaxError: invalid syntax (4050765821.py, line 1)

In [ ]:
## Run Density of States 
!pw.x < pw.scf.silicon_dos.in > pw.scf.silicon_dos.out

In [ ]:
The DOS data in the si_dos.dat file that we specified in our input file

In [ ]:
dos.x < pp.dos.silicon.in > pp.dos.silicon.out

In [ ]:
We can plot the DOS:

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParamsDefault
import numpy as np
%matplotlib inline

# load data
energy, dos, idos = np.loadtxt('../src/silicon/si_dos.dat', unpack=True)

# make plot
plt.figure(figsize = (12, 6))
plt.plot(energy, dos, linewidth=0.75, color='red')
plt.yticks([])
plt.xlabel('Energy (eV)')
plt.ylabel('DOS')
plt.axvline(x=6.642, linewidth=0.5, color='k', linestyle=(0, (8, 10)))
plt.xlim(-6, 16)
plt.ylim(0, )
plt.fill_between(energy, 0, dos, where=(energy < 6.642), facecolor='red', alpha=0.25)
plt.text(6, 1.7, 'Fermi energy', fontsize= med, rotation=90)
plt.show()

In [10]:
### Important
For a set of calculation, we must keep the prefix same. For example, the nscf or 
bands calculation uses the wavefunction calculated by the scf calculation.
When performing different calculations, for example you change a parameter and want to see the changes, you must use different output folder or
unique prefix for different calculations so that the outputs do not get mixed.

SyntaxError: invalid syntax (694368348.py, line 2)

***
### EXERCISE 
<br>
Part 1: Understanding the Script
1- What is the purpose of the occupations = 'tetrahedra' parameter in the &system section?
<br> 
2- What effect does the nosym = .TRUE. option have in the calculation?
  <br>
3- What is the significance of the nbnd parameter, and why is it set to a larger value (e.g., 200) for the NSCF calculation?
<br>
Part 1 : Modify the Number of Bands (nbnd)
<br>
4- Set the nbnd parameter to 300 to ensure that enough bands are considered, especially for higher-lying unoccupied states.
How do you determine the appropriate value for nbnd?


***
## Tutorial Links

* [Click here to open Tutorial 1](LAMMPS-Tutorials-01.ipynb)
* [Click here to open the next tutorial](LAMMPS-Tutorials-03.ipynb)

***
## References 

1. S. Plimpton, "Fast Parallel Algorithms for Short-Range Molecular Dynamics," J. Comp. Phys., 117, 1-19 (1995).